# [Bonus Tasks](svm_segm.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.ndimage as ndi
import skimage.morphology as morph
import skimage.util
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [2]:
def blend_result(img, seg, border_radius=1, interior_opacity=1, interior_opacity_decay=0.9, color=(0,1,0)):
    img  = np.dstack([img] * 3).copy()
    img -= img.min()
    img /= img.max()
    selem  = morph.disk(border_radius)
    seg_bd = np.logical_xor(morph.binary_dilation(seg, selem), morph.binary_erosion(seg, selem))
    mask_decay = ndi.distance_transform_edt(seg)
    for i in range(3):
        opacity = interior_opacity / pow(1 + mask_decay[seg], interior_opacity_decay)
        img[:,:,i][seg] = color[i] * opacity + (1 - opacity) * img[:,:,i][seg]
        img[:,:,i][seg_bd] = color[i]
    return img

In [3]:
patch_sizes = (32, 32)

---

**[Task 1.1.]()** Implement `create_data_matrix`:

In [15]:
def create_data_matrix(img) :
    patch = skimage.util.view_as_blocks(img, block_shape = patch_sizes)
    num_patch = patch.shape[0] * patch.shape[1]
    X = patch.reshape(num_patch, -1)
    
    return np.array(X)

**[Task 1.2.]()** Implement `create_gt_labels_vector`:

In [16]:
def create_gt_labels_vector(img) :
    patch = skimage.util.view_as_blocks(img, block_shape = patch_sizes)
    num_patch = patch.shape[0] * patch.shape[1]
    

    a = []

    for i in range(patch.shape[0]) :
        for j in range(patch.shape[1]) :
            patch = patch[i, j]

            if np.mean(patch) > 0.5 :
                a.append(1)
            
            elif np.mean(patch) == 0 :
                a.append(-1)
            
            else :
                a.append(0)

    return np.array(a)

**[Task 1.3.]()** Create the SVM classifier:

In [17]:
clf = make_pipeline(StandardScaler(), SVC(class_weight='balanced', gamma=0.1))

**[Task 1.3 (a).]()** Create the data matrices for the images `dna-33` and `dna-44`:

In [ ]:
dna33 = plt.imread('data/NIH3T3/im/dna-33.png')
dna44 = plt.imread('data/NIH3T3/im/dna-44.png')

a3 = create_data_matrix(dna33)
a4 = create_data_matrix(dna44)


**[Task 1.3 (b).]()** Create the corresponding ground truth label vectors:

In [23]:
def create_gt_labels_vector(img) :
    h,w = img.shape[:2]
    labels = []

    for i in range(h):
        for j in range(w):
            pixel = img[i,j]
            labels.append(pixel)

    return np.array(labels)        
    

g3 = plt.imread('data/NIH3T3/gt/33.png')
g4 = plt.imread('data/NIH3T3/gt/44.png')

b3 = create_gt_labels_vector(g3)
b4 = create_gt_labels_vector(g4)

print(g3.shape)

(1024, 1344)


**[Task 1.3 (c).]()** Create the *combined* data matrices and ground truth label vectors:

**[Task 1.3 (d).]()** Train the classifier using the data matrix and label vectors from above:

**[Task 1.4.]()** Implement the function `predict_image`:

Test your implementation:

In [ ]:
img = plt.imread(f'data/NIH3T3/im/dna-0.png')
seg = predict_image(img)

plt.figure()
plt.imshow(blend_result(img, seg))

**[Task 1.5.]()** Perform batch processing: